# Contacts in protein chains and calcium ions 

In [1]:
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa
import pandas as pd

## Chain A and Chain B proximity contact analysis 

### The following function will conduct a promiximity analysis. Do not modify. 

In [2]:
def find_contacts(pdb_file, chain1_id="A", chain2_id="B", distance_cutoff=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", pdb_file)
    model = structure[0]

    chain1_atoms = [atom for res in model[chain1_id] if is_aa(res) for atom in res]
    chain2_atoms = [atom for res in model[chain2_id] if is_aa(res) for atom in res]

    ns = NeighborSearch(chain2_atoms)

    chain1_contacts = set()
    chain2_contacts = set()

    for atom in chain1_atoms:
        neighbors = ns.search(atom.coord, distance_cutoff)
        for neighbor in neighbors:
            res1 = atom.get_parent()
            res2 = neighbor.get_parent()
            if res1 != res2:
                chain1_contacts.add((res1.get_resname(), res1.get_id()[1]))
                chain2_contacts.add((res2.get_resname(), res2.get_id()[1]))

    return sorted(chain1_contacts, key=lambda x: x[1]), sorted(chain2_contacts, key=lambda x: x[1])


### The following function saves the data from the above in a dataframe. Do not modify.

In [3]:
import pandas as pd

def format_contacts_as_dataframe(chain1_contacts, chain2_contacts, chain1_id="A", chain2_id="B"):
    # sets to sorted lists
    sorted_a = sorted(chain1_contacts, key=lambda x: x[1])
    sorted_b = sorted(chain2_contacts, key=lambda x: x[1])

    max_len = max(len(sorted_a), len(sorted_b))
    sorted_a += [("", "")] * (max_len - len(sorted_a))
    sorted_b += [("", "")] * (max_len - len(sorted_b))

    # merge into a dataframe
    df = pd.DataFrame({
        f"Chain {chain1_id} Residue": [res[0] for res in sorted_a],
        f"Chain {chain1_id} Position": [res[1] for res in sorted_a],
        f"Chain {chain2_id} Residue": [res[0] for res in sorted_b],
        f"Chain {chain2_id} Position": [res[1] for res in sorted_b],
    })

    return df

### Follow the below instructions to get the contact residues:
1. In the variable 'pdb_path', enter the path to the pdb file you want to analyze. Put it in double quotes i.e. as a python string.
2. In the find_contacts function, put the approproate chain ids you want in the parameters. You can modify the distance_cutoff here as needed.
3. In the format_contacts_as_dataframe function, change "A" and "B" to whatever your chain ids. These should match the chain ids in the find_contacts parameters.
4. Run the cell

In [7]:
# put the path to your pdb file here 
pdb_path = "/Users/jriya/Desktop/-ex7_apoer2_tillex8_reel56_model_0.pdb"
chainA_res, chainB_res = find_contacts(pdb_path, chain1_id="A", chain2_id="B", distance_cutoff=3.5)

df_contacts = format_contacts_as_dataframe(chainA_res, chainB_res, "A", "B")
df_contacts

,Chain A Residue,Chain A Position,Chain B Residue,Chain B Position
0,TRP,22,LYS,234
1,ASP,25,SER,236
2,ASP,27,LYS,238
3,ASP,29,GLU,252
4,ARG,60,ASP,253
5,TRP,61,ASP,285
6,ASP,64,ARG,287
7,GLU,68,PHE,347
8,LYS,89,TYR,348
9,TRP,104,ILE,402


The above gives the contact positons as numbered in the pdb file. These do not reflect the actual positions of amino acids on the full length protein if you have made modifications (eg. removal of signal peptides, splicing exons, etc.) 

### Function to map exon coordinates of ApoER2 to get the positions on the full-length ApoER2 protein. Here's how to use it: 

1. Change the selected_exons list to include the exons you used for the protein. If this step has errors, you can get wrong positions. 
2. Run the cell and the cell following it. Ignore the names of the columns. The output is still for the chains you input in the functions above. 

3. The cell below will display the residues and positions in a dataframe. Uncomment the last line to save as a csv file for every pdb. 
4. "Chain A Position" is for ApoER2 as saved in the pdb file and "Full Position" is the residue number on full length ApoER2 protein.  

In [8]:
# Step 1: Define exon coordinates
exon_coords = {
    1:  (1, 42),   2:  (43, 82),   3:  (83, 123),  4:  (124, 166),  5:  (167, 295),
    6:  (296, 336), 7:  (337, 376), 8:  (377, 418), 9:  (419, 476), 10: (477, 552),
    11: (553, 592), 12: (593, 638), 13: (639, 686), 14: (687, 737), 15: (738, 812),
    16: (813, 835), 17: (836, 892), 18: (893, 951), 19: (952, 963)
}

# Step 2: Specify selected exons used in the spliced construct
selected_exons = [2,3,4,5,6,8]

# Step 3: Build mapping from PDB position → full position and exon
pdb_to_full = {}
pdb_to_spliced = {}
pdb_to_exon = {}
spliced_pos = 1
pdb_pos = 1

for exon in selected_exons:
    start, end = exon_coords[exon]
    for full_pos in range(start, end + 1):
        pdb_to_full[pdb_pos] = full_pos
        pdb_to_spliced[pdb_pos] = spliced_pos
        pdb_to_exon[pdb_pos] = exon
        spliced_pos += 1
        pdb_pos += 1

# Step 4: Identify Chain A columns dynamically
chain_cols = [col for col in df_contacts.columns if "Chain" in col and "Position" in col]
residue_cols = [col for col in df_contacts.columns if "Chain" in col and "Residue" in col]
chain_to_map = chain_cols[0]  # Chain A Position
residue_col = residue_cols[0]  # Chain A Residue
other_cols = [col for col in df_contacts.columns if col not in [chain_to_map, residue_col]]

# Step 5: Clean and map
df_clean = df_contacts.copy()
df_clean = df_clean[df_clean[chain_to_map].apply(lambda x: str(x).isdigit())]
df_clean[chain_to_map] = df_clean[chain_to_map].astype(int)

df_clean["Full Sequence Position"] = df_clean[chain_to_map].map(pdb_to_full)
df_clean["Spliced Position"] = df_clean[chain_to_map].map(pdb_to_spliced)
df_clean["Exon"] = df_clean[chain_to_map].map(pdb_to_exon)

# Step 6: Final cleaned and sorted output
df_contacts_mapped = df_clean.dropna(subset=["Spliced Position"])
df_contacts_mapped = df_contacts_mapped.sort_values("Spliced Position")

# Reorder columns to show main mapping info first
output_cols = [residue_col, chain_to_map, "Full Sequence Position", "Spliced Position", "Exon"] + other_cols
df_contacts_mapped = df_contacts_mapped[output_cols]


In [9]:
df_contacts_mapped

,Chain A Residue,Chain A Position,Full Sequence Position,Spliced Position,Exon,Chain B Residue,Chain B Position
0,TRP,22,64,22,2,LYS,234
1,ASP,25,67,25,2,SER,236
2,ASP,27,69,27,2,LYS,238
3,ASP,29,71,29,2,GLU,252
4,ARG,60,102,60,3,ASP,253
5,TRP,61,103,61,3,ASP,285
6,ASP,64,106,64,3,ARG,287
7,GLU,68,110,68,3,PHE,347
8,LYS,89,131,89,4,TYR,348
9,TRP,104,146,104,4,ILE,402


In [8]:
with pd.option_context('display.max_rows', 100):  # Set to a value >= 61
    display(df_contacts_mapped.head(61))

,Chain A Residue,Chain A Position,Full Sequence Position,Spliced Position,Exon,Chain B Residue,Chain B Position
0,TRP,22,64,22,2,SER,236
1,ASP,25,67,25,2,ARG,237
2,ASP,27,69,27,2,LYS,238
3,ASP,29,71,29,2,SER,363
4,HIS,55,97,55,3,ILE,402
5,TRP,61,103,61,3,LYS,404
6,ASP,64,106,64,3,LYS,511
7,GLU,68,110,68,3,GLN,512
8,LEU,299,341,299,7,ILE,527
9,HIS,307,349,307,7,ASP,528


## Calcium binding analysis 

In [9]:
def find_chain_ion_contacts(pdb_file, chain_id="A", ion_resname="CA", distance_cutoff=3.5):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", pdb_file)
    model = structure[0]

    # Get atoms from the protein chain (only amino acids)
    chain_atoms = [atom for res in model[chain_id] if is_aa(res) for atom in res]

    # Get atoms of the calcium ions anywhere in the structure
    ion_atoms = []
    for chain in model:
        for res in chain:
            if res.get_resname() == ion_resname:
                ion_atoms.extend(res.get_atoms())

    # Neighbor search using ion atoms
    ns = NeighborSearch(ion_atoms)

    contacting_residues = set()

    for atom in chain_atoms:
        neighbors = ns.search(atom.coord, distance_cutoff)
        for neighbor in neighbors:
            res = atom.get_parent()
            contacting_residues.add((res.get_resname(), res.get_id()[1]))

    return sorted(contacting_residues, key=lambda x: x[1])

In [11]:
pdb_path = "/Users/jriya/Desktop/apoer2ex3apoe3rbd4ca.pdb"
ca_contacts = find_chain_ion_contacts(pdb_path, chain_id="A", ion_resname="CA", distance_cutoff=3.5)

print("Residues of the selected chain interacting with specified ion:")
for res in ca_contacts:
    print(f"{res[0]} {res[1]}")

Residues of the selected chain interacting with specified ion:
ASP 6
ASP 8
TRP 21
ASP 24
GLU 26
ASP 34
GLU 35
